## xmlフォーマットの力場ファイルから必要な行を抜き出し.txtファイルにするノートブック

In [83]:
import xml.etree.ElementTree as ET
import itertools
import random
import copy
import re
import os.path

In [73]:
#path
input_dir = "./"
if not os.path.exists(input_dir):
    print("INPUT DIRECTORY IS NOT EXIST")
ff_input_xml_filepath = os.path.join(input_dir, "output.xml") #xmlファイル
ff_input_txt_filepath = os.path.join(input_dir, "output.txt") #txtファイル

output_dir = "./"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
ff_output_xml_filepath = os.path.join(output_dir, "output.xml") #xmlファイル

In [74]:
#atom_type
phi_atom_type = [['C'], ['N'], ['CT'], ['C']]
psi_atom_type = [['N'], ['CT'], ['C'], ['N']]

## input txt file

In [75]:
#txt file
with open(ff_input_txt_filepath, "r") as f:
    datalist = f.readlines()

## テキストファイルの情報を辞書型に変換

In [79]:
#create forcefield
phi_ff_param_updated = dict()
psi_ff_param_updated = dict()

current = phi_ff_param_updated #更新する配列
#phi_ff_param_updatedを更新
for line in datalist:
    if(len(line.split()) < 2 and line.split()[0] == "PHI_PARAMETER"):
        continue
    #PSI_PARAMETERが現れたらcurrent->psi_ff_param_updatedにしてcontinue
    if(len(line.split()) < 2 and line.split()[0] == "PSI_PARAMETER"):
        current = psi_ff_param_updated 
        continue
    #辞書を更新
    elif(line.split()[0] != "PSI_PARAMETER\n"):
        current[line.split()[0]] = line.split()[1]
    else:
        print("ERROR")

## input xml format

In [80]:
#create xml file
tree = copy.deepcopy(ET.parse(ff_input_filepath)) #xml formatのff
root = tree.getroot()

In [82]:
# XMLから'PeriodicTorsionForce'を検索
for torsion in root.iter('PeriodicTorsionForce'):

    #各要素の中から、phiとpsiに関するパラメータを検索
    for ff_local in torsion:
        #print(list(ff_local.attrib.values())[0:4]) [0:4]でスライスすることで、アトムタイプを取り出す
        #phiに関係する二面角のパラメータを取り出す
        if list(ff_local.attrib.values())[0:4] == list(itertools.chain.from_iterable(phi_atom_type)):
            ff_local.attrib = phi_ff_param_updated
            
        #psiに関係する二面角のパラメータを取り出す
        if list(ff_local.attrib.values())[0:4] == list(itertools.chain.from_iterable(psi_atom_type)):        
            ff_local.attrib = psi_ff_param_updated

#output
tree.write(ff_output_xml_filepath)